In [1]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform {USER_FLAG} -q
! pip3 install -U google-cloud-storage {USER_FLAG} -q
! pip3 install {USER_FLAG} kfp google-cloud-pipeline-components --upgrade -q

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cloud-tpu-client 0.10 requires google-api-python-client==1.8.0, but you have google-api-python-client 1.12.11 which is incompatible.


In [1]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [1]:
import kfp
from kfp.v2 import dsl
from kfp.v2.dsl import component
from kfp.v2.dsl import (
    Input,
    Output,
    Artifact,
    Dataset,
)

In [2]:
#The Google Cloud project that this pipeline runs in.
project_id = "de2022-rrd"
# The region that this pipeline runs in
region = "us-central1"
# Specify a Cloud Storage URI that your pipelines service account can access. The artifacts of your pipeline runs are stored within the pipeline root.
pipeline_root_path = "gs://dtemp_a1"

# Pipeline component: Data ingestion

In [3]:
from typing import Dict

def download_data(project_id: str, bucket: str, file_name: str) -> Dict:
    '''download data'''
    from google.cloud import storage
    import pandas as pd
    import logging 
    import sys
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    # Downloaing the file from a google bucket 
    client = storage.Client(project=project_id)
    bucket = client.get_bucket(bucket)
    blob = bucket.blob(file_name)
    local_path = '/tmp/'+ file_name
    blob.download_to_filename(local_path)
    logging.info('Downloaded Data!')
    
    # Convert the data to a dictiory object    
    dict_from_csv = pd.read_csv(local_path, index_col=None, squeeze=True).to_dict()
    logging.info('Returning Data as Dictionary Object!')
    return dict_from_csv

In [4]:
# create a KFP component for data ingestion
data_ingestion_component = kfp.components.create_component_from_func(
    download_data, output_component_file='data_ingestion.yaml', packages_to_install=['google-cloud-storage', 'pandas'])

# Pipeline component: Training SVC

In [7]:
from typing import NamedTuple, Dict

def train_svc (features: Dict, project_id: str, model_repo: str ) -> Dict:
    import pandas as pd
    import numpy as np
    import logging
    import sys
    import os
    import joblib
    from google.cloud import storage
    from sklearn import metrics
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import RobustScaler
    from sklearn.svm import SVC
    from sklearn.metrics import accuracy_score


    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    # reading data
    df = pd.DataFrame.from_dict(features)
    cat_cols = ['sex','exng','caa','cp','fbs','restecg','slp','thall']
    con_cols = ["age","trtbps","chol","thalachh","oldpeak"]

    # creating copy of df and encode categorical columns
    df1 = df
    df1 = pd.get_dummies(df1, columns = cat_cols, drop_first = True)

    # defining features and target
    y = df1[['output']]
    X = df1.drop(['output'],axis=1)

    # instantiate the scaler
    scaler = RobustScaler()

    # scaling the continuous features
    X[con_cols] = scaler.fit_transform(X[con_cols])

    # train test split and fit
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 42)
    clf = SVC(kernel='linear', C=1, random_state=42).fit(X_train,y_train)
    y_pred = clf.predict(X_test)

    # Accuracy logging
    metrics = {
        "accuracy": accuracy_score(y_test, y_pred)
    }
    logging.info(metrics)

    # Save the model localy
    local_file = '/tmp/local_model.h5'
    joblib.dump(clf, local_file)

    # Save to GCS as model.pkl
    client = storage.Client(project=project_id)
    bucket = client.get_bucket(model_repo)
    blob = bucket.blob('svc_model.h5')
    # Upload the locally saved model
    blob.upload_from_filename(local_file)

    print("Saved the model to GCP bucket : " + model_repo)
    return metrics

In [8]:
# create a KFP component for training SVC model
train_svc_component = kfp.components.create_component_from_func(
    train_svc, output_component_file='train_svc_model.yaml', packages_to_install=['google-cloud-storage', 'tensorflow' ,'pandas', 'joblib', 'scikit-learn'])

# Pipeline Component: Logistic regression

In [11]:
from typing import NamedTuple, Dict

def train_LR (features: Dict, project_id: str, model_repo: str ) -> Dict:
    import pandas as pd
    import numpy as np
    import logging
    import sys
    import os
    import joblib
    from google.cloud import storage
    from sklearn import metrics
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import RobustScaler
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import accuracy_score


    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    # reading data
    df = pd.DataFrame.from_dict(features)
    cat_cols = ['sex','exng','caa','cp','fbs','restecg','slp','thall']
    con_cols = ["age","trtbps","chol","thalachh","oldpeak"]

    # creating copy of df and encode categorical columns
    df1 = df
    df1 = pd.get_dummies(df1, columns = cat_cols, drop_first = True)

    # defining features and target
    y = df1[['output']]
    X = df1.drop(['output'],axis=1)

    # instantiate the scaler
    scaler = RobustScaler()

    # scaling the continuous features
    X[con_cols] = scaler.fit_transform(X[con_cols])

    # train test split and fit
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 42)
    
    # instantiating the object
    logreg = LogisticRegression()

    # fitting the object
    logreg.fit(X_train, y_train)
    
    # calculating the probabilities
    y_pred_proba = logreg.predict_proba(X_test)

    # finding the predicted valued
    y_pred = np.argmax(y_pred_proba,axis=1)

    # Accuracy logging
    metrics = {
        "accuracy": accuracy_score(y_test, y_pred)
    }
    logging.info(metrics)

    # Save the model localy
    local_file = '/tmp/local_model.h5'
    joblib.dump(logreg, local_file)

    # Save to GCS as model.pkl
    client = storage.Client(project=project_id)
    bucket = client.get_bucket(model_repo)
    blob = bucket.blob('logreg_model.h5')
    # Upload the locally saved model
    blob.upload_from_filename(local_file)

    print("Saved the model to GCP bucket : " + model_repo)
    return metrics

In [12]:
# create a KFP component for training logreg model
train_lr_component = kfp.components.create_component_from_func(
    train_LR, output_component_file='train_LR_model.yaml', packages_to_install=['google-cloud-storage', 'pandas', 
                                                                                'joblib', 'scikit-learn'])

# Model comparison component

In [13]:
from typing import NamedTuple, Dict

def compare_model_result(svc_metrics: Dict, lr_metrics: Dict) -> str:
    import logging
    import json
    import sys
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    logging.info(svc_metrics)
    logging.info(lr_metrics)
    if svc_metrics.get("accuracy") > lr_metrics.get("accuracy"):
        return "SVC"
    else :
        return "LR"

In [14]:
# component for selecting between SVC and LR
compare_model_comp = kfp.components.create_component_from_func(
    compare_model_result, output_component_file='model_comparison_result.yaml')

# Define pipeline

In [15]:
@kfp.dsl.pipeline(
    name="heart-attack-training-pipeline",
    pipeline_root=pipeline_root_path)

def pipeline(project_id: str, data_bucket: str, dataset_filename: str, model_repo: str, ):
    
    data_ingestion_op = data_ingestion_component(
    project_id = project_id,
    bucket = data_bucket,
    file_name = dataset_filename
    )
    
    training_svc_job_run_op = train_svc_component(
    project_id = project_id,
    model_repo = model_repo,
    features = data_ingestion_op.output
    )
    
    training_lr_job_run_op = train_lr_component(
    project_id = project_id,
    model_repo = model_repo,
    features = data_ingestion_op.output
    )
    
    compare_model_op = compare_model_comp(training_svc_job_run_op.output,
                                   training_lr_job_run_op.output).after(training_svc_job_run_op, training_lr_job_run_op)

# Compile pipeline to JSON

In [16]:
from kfp.v2 import compiler
compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='heart_attack_training_pipeline.json')

/home/jupyter/.local/lib/python3.7/site-packages/kfp/v2/compiler/compiler.py:1293: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,


# Submit pipeline run

In [17]:
import google.cloud.aiplatform as aip

job = aip.PipelineJob(
    display_name="heart_attack_training_pipeline",
    enable_caching=False,
    template_path="heart_attack_training_pipeline.json",
    pipeline_root=pipeline_root_path,
    parameter_values={
        'project_id': project_id, 
        'data_bucket': 'data_a1',  
        'dataset_filename': 'heart.csv',     
        'model_repo':'dmodel_a1'
    }
)

job.run()

Creating PipelineJob
PipelineJob created. Resource name: projects/539022204835/locations/us-central1/pipelineJobs/heart-attack-training-pipeline-20221025201300
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/539022204835/locations/us-central1/pipelineJobs/heart-attack-training-pipeline-20221025201300')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/heart-attack-training-pipeline-20221025201300?project=539022204835
PipelineJob projects/539022204835/locations/us-central1/pipelineJobs/heart-attack-training-pipeline-20221025201300 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/539022204835/locations/us-central1/pipelineJobs/heart-attack-training-pipeline-20221025201300 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/539022204835/locations/us-central1/pipelineJobs/heart-attack-training-pipeline-20221025201300 current state:
PipelineState.PIPE